In [1]:
import pandas as pd
import pandas as pd
import nltk
import string 
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [620]:
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import snowball
from nltk.stem import WordNetLemmatizer
import re
import string

In [621]:
df= pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [622]:
df.shape

(7613, 5)

In [623]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [624]:
df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [625]:
# forming dataframe by eliminating missing keyword values
df1= df.copy()
df1= df1[~df1['keyword'].isna()]
df1.reset_index(inplace=True)

In [626]:
df1.head()

,index,id,keyword,location,text,target
0,31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
2,33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
4,35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [627]:
df1.shape

(7552, 6)

In [628]:
df1.isnull().sum()

index          0
id             0
keyword        0
location    2472
text           0
target         0
dtype: int64

In [629]:
# top ten frequently used keywords
df1['keyword'].value_counts().head(10)

fatalities     45
armageddon     42
deluge         42
body%20bags    41
sinking        41
harm           41
damage         41
outbreak       40
fear           40
siren          40
Name: keyword, dtype: int64

In [630]:
df1['id'].value_counts()

2049     1
8753     1
589      1
8777     1
10824    1
        ..
3299     1
1250     1
7393     1
5344     1
2047     1
Name: id, Length: 7552, dtype: int64

In [631]:
df1.duplicated(subset=['id'], keep='first')
df1.shape

(7552, 6)

In [632]:
df2= df1.copy()
df2.drop(['index', 'id', 'location'],axis=1, inplace=True)

In [633]:
df2.columns

Index(['keyword', 'text', 'target'], dtype='object')

In [634]:
df1[['text','target']].groupby(['target']).describe()

text                                                               
       count unique                                                top freq
target                                                                     
0       4323   4296  He came to a land which was engulfed in tribal...    4
1       3229   3169  11-Year-Old Boy Charged With Manslaughter of T...   10

In [635]:
df1['text'][1]

'We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw'

In [636]:
df1['text'][0]

'@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C'

In [637]:
df1['text'][2]

'#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi'

In [638]:
df1['text'][3]

'Crying out for more! Set me ablaze'

In [639]:
df1['text'][4]

'On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N'

In [640]:
df1['text'][5]

"@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season."

In [641]:
df2.head()

,keyword,text,target
0,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,ablaze,We always try to bring the heavy. #metal #RT h...,0
2,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,ablaze,Crying out for more! Set me ablaze,0
4,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [642]:
df2['text_keyword']= np.nan
for row in range(0, len(df2.index)):
    df2['text_keyword'][row]= ' '.join(str(x) for x in df2.iloc[row, 0:2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [643]:
df2.head()

,keyword,text,target,text_keyword
0,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,ablaze @bbcmtd Wholesale Markets ablaze http:/...
1,ablaze,We always try to bring the heavy. #metal #RT h...,0,ablaze We always try to bring the heavy. #meta...
2,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,ablaze #AFRICANBAZE: Breaking news:Nigeria fla...
3,ablaze,Crying out for more! Set me ablaze,0,ablaze Crying out for more! Set me ablaze
4,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,ablaze On plus side LOOK AT THE SKY LAST NIGHT...


In [644]:
df2= df2.drop(['text'], axis=1)
df2.head()

,keyword,target,text_keyword
0,ablaze,1,ablaze @bbcmtd Wholesale Markets ablaze http:/...
1,ablaze,0,ablaze We always try to bring the heavy. #meta...
2,ablaze,1,ablaze #AFRICANBAZE: Breaking news:Nigeria fla...
3,ablaze,0,ablaze Crying out for more! Set me ablaze
4,ablaze,0,ablaze On plus side LOOK AT THE SKY LAST NIGHT...


In [645]:
stem= PorterStemmer()
lem= WordNetLemmatizer()
def text_process(mess):
    no_url = re.sub(r"http\S+", "", mess)
    no_punc= [char for char in no_url if char not in string.punctuation]
    no_punc= ''.join(no_punc)
    tweet= re.sub('[^A-Za-z]',' ',no_punc)
    tweet= tweet.lower()
    return [lem.lemmatize(word) for word in tweet.split() if word not in stopwords.words('english')]

In [607]:
df2['text_keyword'].head(5).apply(text_process)

0          [ablaze, bbcmtd, wholesale, market, ablaze]
1       [ablaze, always, try, bring, heavy, metal, rt]
2    [ablaze, africanbaze, breaking, newsnigeria, f...
3                           [ablaze, cry, set, ablaze]
4    [ablaze, plus, side, look, sky, last, night, a...
Name: text_keyword, dtype: object

In [608]:
df2['text_keyword']= df2['text_keyword'].apply(text_process)

In [646]:
df2['text_keyword'][0]

'ablaze @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C'

In [647]:
df2['text_keyword'][1]

'ablaze We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw'

In [648]:
df2['text_keyword'][2]

'ablaze #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi'

In [649]:
df2['text_keyword'][3]

'ablaze Crying out for more! Set me ablaze'

In [650]:
df3=df2.drop(['keyword'], axis=1)
df3.head()

,target,text_keyword
0,1,ablaze @bbcmtd Wholesale Markets ablaze http:/...
1,0,ablaze We always try to bring the heavy. #meta...
2,1,ablaze #AFRICANBAZE: Breaking news:Nigeria fla...
3,0,ablaze Crying out for more! Set me ablaze
4,0,ablaze On plus side LOOK AT THE SKY LAST NIGHT...


In [651]:
df3.tail()

,target,text_keyword
7547,0,wrecked @jt_ruff23 @cameronhacker and I wrecke...
7548,0,wrecked Three days off from work and they've p...
7549,0,wrecked #FX #forex #trading Cramer: Iger's 3 w...
7550,0,wrecked @engineshed Great atmosphere at the Br...
7551,0,wrecked Cramer: Iger's 3 words that wrecked Di...


In [704]:
df3['target'].value_counts()

0    4323
1    3229
Name: target, dtype: int64

In [679]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
count_vectorizer= CountVectorizer(analyzer=text_process)
feature_vector= count_vectorizer.fit(df3.text_keyword)
train_ds_features= count_vectorizer.transform(df3.text_keyword)

In [680]:
train_ds_features

<7552x15358 sparse matrix of type '<class 'numpy.int64'>'
	with 67986 stored elements in Compressed Sparse Row format>

In [681]:
train_ds_features.shape

(7552, 15358)

In [682]:
# matrix stored as sparse matrix!-->most of the ele will be 0
# no.of non-zero values in the matrix
train_ds_features.getnnz()

67986

In [683]:
#propotion of non-zero values w.r.t zero values in the matrix
print('Density of matrix:',
     train_ds_features.getnnz()*100/(train_ds_features.shape[0]*train_ds_features.shape[1]))

Density of matrix: 0.05861689982143685


In [684]:
# matrix has less than 1% non-zero values
# 99% zero values
# This is a very sparse representation

In [685]:
features= feature_vector.get_feature_names()

In [686]:
features[:10]

['aa',
 'aaaa',
 'aaaaaaallll',
 'aaaaaand',
 'aaarrrgghhh',
 'aaceorg',
 'aal',
 'aampb',
 'aampw',
 'aan']

In [687]:
# count the frequency of features
feature_counts= np.sum(train_ds_features.toarray(), axis=0)
feature_counts= pd.DataFrame(dict(features= features, counts= feature_counts))

In [688]:
feature_counts

,features,counts
0,aa,1
1,aaaa,1
2,aaaaaaallll,1
3,aaaaaand,1
4,aaarrrgghhh,1
...,...,...
15353,zrnf,1
15354,zumiez,1
15355,zurich,1
15356,zxathetis,1


In [689]:
# top 15 frequent words used in tweets
feature_counts.sort_values(by='counts', ascending=False)[:20]

,features,counts
4908,fire,558
7832,like,346
4256,emergency,304
468,amp,302
6545,im,298
3683,disaster,259
1544,body,258
5444,get,255
14143,u,254
1818,building,245


In [664]:
train_ds_df= pd.DataFrame(train_ds_features.todense())
train_ds_df.columns= features

In [665]:
train_ds_df.head()

,aa,aaaa,aaaaaaallll,aaaaaand,aaarrrgghhh,aaceorg,aal,aampb,aampw,aan,...,zonewolf,zoom,zotar,zouma,zourryart,zrnf,zumiez,zurich,zxathetis,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [691]:
# analysing and removing low frequency words
#import matplotlib.pyplot as plt
#plt.figure(figsize=(12,5))
#plt.hist(train_ds_df, bins=50, range=(0,2000));
#plt.xlabel('Freq of words')
#plt.ylabel('Density')

In [690]:
len(feature_counts[feature_counts.counts==1])

9520

In [533]:
# some words like im, u, via seems irrelevent

In [672]:
from sklearn.feature_extraction import text
my_stop_words= text.ENGLISH_STOP_WORDS
print(list(my_stop_words)[:10])

['call', 'enough', 'or', 'much', 'with', 'throughout', 'thereupon', 'whence', 'however', 'all']


In [673]:
my_stop_words_new= text.ENGLISH_STOP_WORDS.union(['im', 'u'])
my_stop_words_new

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [718]:
count_vectorizer= CountVectorizer(analyzer=text_process,stop_words=my_stop_words_new,max_features=5800,ngram_range=(1,1))
feature_vector= count_vectorizer.fit(df3.text_keyword)
train_ds_features= count_vectorizer.transform(df3.text_keyword)
features= feature_vector.get_feature_names()
feature_counts= np.sum(train_ds_features.toarray(), axis=0)
feature_counts= pd.DataFrame(dict(features= features, counts= feature_counts))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:502: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


In [719]:
feature_counts.sort_values(by='counts', ascending=False)[:20]

,features,counts
1892,fire,558
2916,like,346
1596,emergency,304
164,amp,302
2479,im,298
1393,disaster,259
571,body,258
2087,get,255
5342,u,254
676,building,245


In [720]:
feature_counts

,features,counts
0,ab,2
1,aba,14
2,abandon,3
3,abandoned,10
4,abbott,2
...,...,...
5795,zippednews,2
5796,zipper,2
5797,zombie,7
5798,zone,56


In [721]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(train_ds_features, df3['target'], test_size=0.2)

In [722]:
from sklearn.naive_bayes import BernoulliNB
nb= BernoulliNB()
nb.fit(X_train, y_train)
y_pred= nb.predict(X_test)

In [724]:
from sklearn.metrics import classification_report, confusion_matrix
report= classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.88      0.83       874
           1       0.81      0.69      0.74       637

    accuracy                           0.80      1511
   macro avg       0.80      0.78      0.79      1511
weighted avg       0.80      0.80      0.80      1511



In [725]:
print(confusion_matrix(y_test, y_pred))

[[768 106]
 [198 439]]
